In [2]:
import pandas as pd
pep = pd.read_csv("/Users/kbhaskar/Desktop/GEO database/peptide-mhc-binding-affinity.csv", index_col=None)
pep.head()

,allele,peptide,measurement_value,measurement_inequality,measurement_type,measurement_kind,measurement_source,original_allele
0,BoLA-1*21:01,AENDTLVVSV,7817.0,=,quantitative,affinity,Barlow - purified MHC/competitive/fluorescence,BoLA-1*02101
1,BoLA-1*21:01,NQFNGGCLLV,1086.0,=,quantitative,affinity,Barlow - purified MHC/direct/fluorescence,BoLA-1*02101
2,BoLA-2*08:01,AAHCIHAEW,21.0,=,quantitative,affinity,Barlow - purified MHC/direct/fluorescence,BoLA-2*00801
3,BoLA-2*08:01,AAKHMSNTY,1299.0,=,quantitative,affinity,Barlow - purified MHC/direct/fluorescence,BoLA-2*00801
4,BoLA-2*08:01,DSYAYMRNGW,2.0,=,quantitative,affinity,Barlow - purified MHC/direct/fluorescence,BoLA-2*00801


In [3]:
#only take human hla alleles
for index, row in pep.iterrows():
    if not row['allele'].startswith("HLA"):
        pep.drop(index, inplace=True)
print(len(pep))

187669


In [4]:
pep.head()

,allele,peptide,measurement_value,measurement_inequality,measurement_type,measurement_kind,measurement_source,original_allele
15886,HLA-A*01:01,AADFPGIAR,20000.0,>,quantitative,affinity,Buus - purified MHC/direct/fluorescence,HLA-A*01:01
15887,HLA-A*01:01,AADKAAAAAY,45.0,=,quantitative,affinity,Michel - purified MHC/competitive/radioactivity,HLA-A*01:01
15888,HLA-A*01:01,AADKAAAAY,50.0,=,quantitative,affinity,Sette - purified MHC/competitive/radioactivity,HLA-A*01:01
15889,HLA-A*01:01,AADSFATSY,76.3,=,quantitative,affinity,Buus - purified MHC/direct/fluorescence,HLA-A*01:01
15890,HLA-A*01:01,AAELDRTEEY,5000.0,>,qualitative,affinity,Jr - purified MHC/direct/fluorescence,HLA-A*01:01


In [5]:
pep = pep.reset_index()
pep = pep.drop('index', 1)

In [6]:
pep.head()

,allele,peptide,measurement_value,measurement_inequality,measurement_type,measurement_kind,measurement_source,original_allele
0,HLA-A*01:01,AADFPGIAR,20000.0,>,quantitative,affinity,Buus - purified MHC/direct/fluorescence,HLA-A*01:01
1,HLA-A*01:01,AADKAAAAAY,45.0,=,quantitative,affinity,Michel - purified MHC/competitive/radioactivity,HLA-A*01:01
2,HLA-A*01:01,AADKAAAAY,50.0,=,quantitative,affinity,Sette - purified MHC/competitive/radioactivity,HLA-A*01:01
3,HLA-A*01:01,AADSFATSY,76.3,=,quantitative,affinity,Buus - purified MHC/direct/fluorescence,HLA-A*01:01
4,HLA-A*01:01,AAELDRTEEY,5000.0,>,qualitative,affinity,Jr - purified MHC/direct/fluorescence,HLA-A*01:01


In [7]:
#normalize MHC IC50 scores using math.log to obtain a score in a scale of 0 to 1. The epitopes with the higer score will be used
#smaller IC50s are more favorable so 1 − log(min(IC50, 50000))/log(50000) was used
import math
listz = list(pep['measurement_value'])
for index, row in pep.iterrows():
    if listz[index] != 0:
        yeet = math.log(min(listz[index], 50000))/math.log(50000)
        if not (1-yeet) > 0.35:
            pep.drop(index, inplace=True)
print(len(pep))

187669
59266


In [14]:
pep = pep["peptide"]

In [11]:
#huge dataset of more epitopes, find an overlap between this one and the previous one
big = pd.read_csv("/Users/kbhaskar/Desktop/GEO database/eluted-mhc-ligands-mass-spec.csv", index_col=None)

/Users/kbhaskar/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [12]:
big = big["peptide"]

In [13]:
big.head()

0    TPSLVKSTSQL
1      RPSGPGPEL
2     SPKPFTEEVL
3      APTLGPAVI
4      KIYEGQVEV
Name: peptide, dtype: object

In [21]:
overlap = pd.concat([big,pep])

In [22]:
len(overlap)

712012

In [27]:
overlap = overlap.drop_duplicates(keep = "first")

In [28]:
len(overlap)

295411

In [29]:
overlap.to_csv("overlap.csv")